In [ ]:
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

import optuna

#### **Getting training & validation data**

In [ ]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [ ]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

In [ ]:
X_train = np.array(X_train)
X_val = np.array(X_val)

#### **Parameters selection**

In [ ]:
params = {
    'hidden_layer_units': [32, 128, 256],
    'hidden_layers': {
        'low': 1,
        'high': 2
    },
    'alpha': [0.0001, 0.01],
    'max_iter': [2000, 5000, 10000],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid'],
    'optimizer': ['SGD', 'Adam'],
    'output_activation': ['sigmoid', 'softmax'],
    'loss_function': ['binary_crossentropy', 'mse'],
    'dropout_rate': [0.5, 0.3, 0.02]
}

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
    min_delta=0.001
)

def objective(trial):
    model = Sequential()
    input_dimension = X_train.shape[1]
    
    model.add(
        Dense(
            input_dim=input_dimension,
            units=trial.suggest_categorical(
                'hidden_layer_units',
                params['hidden_layer_units']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        )
    )

    hidden_layers = trial.suggest_int('hidden_layers', params['hidden_layers']['low'], params['hidden_layers']['high'])
    
    for _ in range(hidden_layers):
        model.add(
            Dense(
                units=trial.suggest_categorical(
                    'hidden_layer_units',
                    params['hidden_layer_units']
                ),
                activation=trial.suggest_categorical(
                    'activation',
                    params['activation']
                )
            )
        )

        model.add(
            Dropout(
                trial.suggest_categorical(
                    'dropout_rate',
                    params['dropout_rate']
                )
            )
        )
    
    model.add(
        Dense(
            units=1,
            activation=trial.suggest_categorical(
                'output_activation',
                params['output_activation']
            )
        )
    )

    alpha = trial.suggest_categorical('alpha', params['alpha'])
    optimizer_name = trial.suggest_categorical('optimizer', params['optimizer'])

    if optimizer_name == 'SGD':
        optimizer = SGD(learning_rate=alpha)
    elif optimizer_name == 'Adam':
        optimizer = Adam(learning_rate=alpha)

    model.compile(
        optimizer=optimizer,
        loss=trial.suggest_categorical('loss_function', params['loss_function']),
        metrics=['accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return loss

In [ ]:
n_trials = 150

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

#### **Saving study**

In [ ]:
import joblib

In [ ]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)